First, create the data generator class that is a child of the tensorflow sequence class.

In [1]:
import keras
import numpy as np


class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.loadtxt('F:\\training3000\\64bit_arrs\\' + ID + '.txt', dtype=np.float16).reshape((128, 128, 1))

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)


Next, create instances of the generator class for training and validation.

In [2]:
import os
from keras import layers
from keras import models
from keras import optimizers
rng = np.random.default_rng()

path = "F:\\training3000\\64bit_arrs"

sample_arr = np.array(os.listdir(path))
c = 0
for i in sample_arr:
    sample_arr[c] = i[:-4]
    c += 1

indices = np.arange(0, sample_arr.size)
rng.shuffle(indices)

frac = 0.2
partition = {"train": sample_arr[indices[int(indices.size*frac):]], "val": sample_arr[indices[:int(indices.size*frac)]]}
labels = {}
for i in sample_arr:
    labels[i] = int((int(i[-4:])-50)/200)

params = {'dim': (128,128),
          'batch_size': 64,
          'n_classes': 10,
          'n_channels': 1,
          'shuffle': True}

training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['val'], labels, **params)


Finally, create and train a model on multiple threads

In [ ]:

model = models.Sequential()
model.add(layers.SeparableConv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu', kernel_regularizer='l2'))
model.add(layers.Dense(10, activation='softmax', kernel_regularizer='l2'))

model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['acc'])

if __name__ == '__main__':
    model.fit_generator(generator=training_generator,
                        validation_data=validation_generator,
                        use_multiprocessing=True,
                        workers=4)

Instructions for updating:
Please use Model.fit, which supports generators.
